In [81]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

In [82]:
df = pd.read_csv("../../data/preprocessed_AQI_data.csv")
df.head()

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,lat,lng
0,130,10126,51,2,1,0,36,0,0,0,51,2,44.7444,44.2031
1,22,10140,41,0,1,0,5,0,1,0,41,0,-5.2900,-44.4900
2,75,10163,66,2,1,0,39,0,2,0,66,2,37.1667,15.1833
3,123,10185,34,0,1,0,34,0,0,0,20,0,53.0167,20.8833
4,166,10243,54,2,1,0,14,0,11,0,54,2,16.1005,-88.8074


In [83]:
from sklearn.model_selection import train_test_split

# shuffle the data before splitting
df_shuffled = df.sample(frac=1, random_state=19)
X = df_shuffled.drop(["AQI Category"], axis=1)
y = df_shuffled["AQI Category"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.8, random_state=22
)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(2845, 13) (11384, 13)
(2845,) (11384,)


In [84]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

mm = MinMaxScaler()
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.8, random_state=22
)

In [85]:
lda_classifier = LinearDiscriminantAnalysis()
lda_classifier.fit(X_train, y_train)
y_pred = lda_classifier.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      5301
           1       1.00      0.88      0.94        41
           2       1.00      0.97      0.98      4741
           3       0.94      0.97      0.96       618
           4       0.98      0.96      0.97       588
           5       0.90      0.85      0.88        95

    accuracy                           0.98     11384
   macro avg       0.97      0.94      0.95     11384
weighted avg       0.98      0.98      0.98     11384



In [86]:
from sklearn.model_selection import GridSearchCV

lda_classifier = LinearDiscriminantAnalysis()
param_grid = {
    "solver": ["svd", "lsqr", "eigen"],
    "shrinkage": ["auto", None],
}

In [87]:
import warnings

warnings.filterwarnings("ignore")

In [88]:
grid_search = GridSearchCV(estimator=lda_classifier, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LinearDiscriminantAnalysis(),
             param_grid={'shrinkage': ['auto', None],
                         'solver': ['svd', 'lsqr', 'eigen']})

In [89]:
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'shrinkage': None, 'solver': 'svd'}


In [90]:
best_lda = LinearDiscriminantAnalysis(shrinkage=None, solver="svd")
best_lda.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [91]:
y_pred = best_lda.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      5301
           1       1.00      0.88      0.94        41
           2       1.00      0.97      0.98      4741
           3       0.94      0.97      0.96       618
           4       0.98      0.96      0.97       588
           5       0.90      0.85      0.88        95

    accuracy                           0.98     11384
   macro avg       0.97      0.94      0.95     11384
weighted avg       0.98      0.98      0.98     11384



Best Parameters: {'shrinkage': None, 'solver': 'svd'}


In [92]:
from sklearn.metrics import accuracy_score

lda_classifier_best = LinearDiscriminantAnalysis(shrinkage=None, solver="svd")
lda_classifier_best.fit(X_train, y_train)
y_pred = lda_classifier_best.predict(X_test)
cr = classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      5301
           1       1.00      0.88      0.94        41
           2       1.00      0.97      0.98      4741
           3       0.94      0.97      0.96       618
           4       0.98      0.96      0.97       588
           5       0.90      0.85      0.88        95

    accuracy                           0.98     11384
   macro avg       0.97      0.94      0.95     11384
weighted avg       0.98      0.98      0.98     11384



In [93]:
acc = accuracy_score(y_pred, y_test)
acc

0.982167955024596

In [94]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (
    make_scorer,
    f1_score,
    precision_score,
    recall_score,
    accuracy_score,
)

# here we define different scorers metrics
scorer_f1 = make_scorer(f1_score, average="weighted")
scorer_precision = make_scorer(precision_score, average="weighted", zero_division=1)
scorer_recall = make_scorer(recall_score, average="weighted")
scorer_accuracy = make_scorer(accuracy_score)

# Cwe save them in a dict for easier accessing
scorers = {
    "f1": scorer_f1,
    "precision": scorer_precision,
    "accuracy": scorer_accuracy,
    "recall": scorer_recall,
}
scores_results: dict = {}
for scorer_name, scorer in scorers.items():
    scores = cross_val_score(best_lda, X_train, y_train, cv=7, scoring=scorer)
    print(f"{scorer_name.capitalize()} scores:", scores)
    scores_results[scorer_name] = round(scores.mean(), 4)
    print(f"{scorer_name.capitalize()} mean:", round(scores.mean(), 4), "\n")

F1 scores: [0.96779886 0.98521533 0.99016911 0.99260415 0.97175013 0.99014018
 0.97197757]
F1 mean: 0.9814 

Precision scores: [0.97163487 0.98550098 0.99022163 0.99272223 0.97458605 0.99036683
 0.9740353 ]
Precision mean: 0.9827 

Accuracy scores: [0.96805897 0.98525799 0.99017199 0.99261084 0.9729064  0.99014778
 0.9729064 ]
Accuracy mean: 0.9817 

Recall scores: [0.96805897 0.98525799 0.99017199 0.99261084 0.9729064  0.99014778
 0.9729064 ]
Recall mean: 0.9817 



In [95]:
import pandas as pd

scores_df = pd.DataFrame.from_dict(scores_results, orient="index", columns=["Scores"])
scores_df

,Scores
f1,0.9814
precision,0.9827
accuracy,0.9817
recall,0.9817
